Siddhartha Chintalapalli




Task 1 - Following URL: http://eecs.qmul.ac.uk/~emmanouilb/income_table.html. This webpage includes a table on individuals' income and shopping habits.

Using Beautiful Soup, scrape the table and convert it into a pandas dataframe. Perform data cleaning when necessary to remove extra characters (no need to handle missing values). In the report include the code that was used to scrape and convert the table and provide evidence that the table has been successfully scraped and converted.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
url = "http://eecs.qmul.ac.uk/~emmanouilb/income_table.html"
html = urlopen(url)

In [3]:
soup = BeautifulSoup(html, 'lxml')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [4]:
header_list = []

# Find the 'th' html tags which denote table header
col_labels = soup.find_all('th')
col_str = str(col_labels)
cleantext_header = BeautifulSoup(col_str, "lxml").get_text()  # extract the text without HTML tags
header_list.append(cleantext_header) # Add the clean table header to the list

print(header_list)

['[Region, Age, Income, Online Shopper]']


In [5]:
df_header = pd.DataFrame(header_list)
df_header.head()

,0
0,"[Region, Age, Income, Online Shopper]"


In [6]:
df_header2 = df_header[0].str.split(',', expand=True)
df_header2.head()

,0,1,2,3
0,[Region,Age,Income,Online Shopper]


In [7]:
df_header2[0] = df_header2[0].str.strip('[')
df_header2[3] = df_header2[3].str.strip(']')

In [8]:
rows = soup.find_all('tr')  # the 'tr' tag in html denotes a table row
print(rows[:10])

[<tr><th title="Field #1">Region</th>
<th title="Field #2">Age</th>
<th title="Field #3">Income</th>
<th title="Field #4">Online Shopper</th>
</tr>, <tr>
<td>India</td>
<td align="right">49</td>
<td align="right">86400</td>
<td>No</td>
</tr>, <tr>
<td>Brazil</td>
<td align="right">32</td>
<td align="right">57600</td>
<td>Yes</td>
</tr>, <tr>
<td>USA</td>
<td align="right">35</td>
<td align="right">64800</td>
<td>No</td>
</tr>, <tr>
<td>Brazil</td>
<td align="right">43</td>
<td align="right">73200</td>
<td>No</td>
</tr>, <tr>
<td>USA</td>
<td align="right">45</td>
<td align="right"></td>
<td>Yes</td>
</tr>, <tr>
<td>India</td>
<td align="right">40</td>
<td align="right">69600</td>
<td>Yes</td>
</tr>, <tr>
<td>Brazil</td>
<td align="right"></td>
<td align="right">62400</td>
<td>No</td>
</tr>, <tr>
<td>India</td>
<td align="right">53</td>
<td align="right">94800</td>
<td>Yes</td>
</tr>, <tr>
<td>USA</td>
<td align="right">55</td>
<td align="right">99600</td>
<td>No</td>
</tr>]


In [9]:
# Create an empty list where the table will be stored
table_list = []

# For every row in the table, find each cell element and add it to the list
for row in rows:
    row_td = row.find_all('td')
    row_cells = str(row_td)
    row_cleantext = BeautifulSoup(row_cells, "lxml").get_text()  # extract the text without HTML tags
    table_list.append(row_cleantext)  # Add the clean table row to the list
    
print(table_list)

['[]', '[India, 49, 86400, No]', '[Brazil, 32, 57600, Yes]', '[USA, 35, 64800, No]', '[Brazil, 43, 73200, No]', '[USA, 45, , Yes]', '[India, 40, 69600, Yes]', '[Brazil, , 62400, No]', '[India, 53, 94800, Yes]', '[USA, 55, 99600, No]', '[India, 42, 80400, Yes]']


In [10]:
df_table = pd.DataFrame(table_list)
df_table2 = df_table[0].str.split(',', expand=True)
df_table2.head(10)

,0,1,2,3
0,[],None,None,None
1,[India,49,86400,No]
2,[Brazil,32,57600,Yes]
3,[USA,35,64800,No]
4,[Brazil,43,73200,No]
5,[USA,45,,Yes]
6,[India,40,69600,Yes]
7,[Brazil,,62400,No]
8,[India,53,94800,Yes]
9,[USA,55,99600,No]


In [11]:
# Remove uneccesary characters
df_table2[0] = df_table2[0].str.strip('[')
df_table2[0] = df_table2[0].str.strip(']')
df_table2[3] = df_table2[3].str.strip(']')

# Remove all rows with any missing values
df_table3 = df_table2.dropna(axis=0, how='any')

df_table3.head(10)

,0,1,2,3
1,India,49,86400,No
2,Brazil,32,57600,Yes
3,USA,35,64800,No
4,Brazil,43,73200,No
5,USA,45,,Yes
6,India,40,69600,Yes
7,Brazil,,62400,No
8,India,53,94800,Yes
9,USA,55,99600,No
10,India,42,80400,Yes


In [12]:
frames = [df_header2, df_table3]
df = pd.concat(frames)

df2 = df.rename(columns=df.iloc[0]) # We assign the first row to be the dataframe header
df3 = df2.drop(df2.index[0]) # We drop the replicated header from the first row of the dataframe

df3.head(10)

,Region,Age,Income,Online Shopper
1,India,49,86400,No
2,Brazil,32,57600,Yes
3,USA,35,64800,No
4,Brazil,43,73200,No
5,USA,45,,Yes
6,India,40,69600,Yes
7,Brazil,,62400,No
8,India,53,94800,Yes
9,USA,55,99600,No
10,India,42,80400,Yes


Task 2- The list of the various MSc programmes offered by the School of EECS is provided at the following URL: http://eecs.qmul.ac.uk/postgraduate/programmes/. Perform web scraping on the table present in the above URL and convert it into a pandas dataframe that would include one row for each programme of study as shown in the webpage. The dataframe should include the following 5 columns: name of postgraduate degree programme (e.g. Advanced Electronic and Electrical Engineering), programme code for part-time study (e.g. H60C), programme code for full-time study (e.g. H60A), URL for part-time study programme details, and URL for full-time study programme details. Perform data cleaning to remove unecessary characters when needed. In the report include the code that was used to scrape, convert and clean the table and provide evidence that the table has been successfully scraped (e.g. by displaying the contents of the dataframe

In [13]:
url = "http://eecs.qmul.ac.uk/postgraduate/programmes/"
html = urlopen(url)

In [14]:
soup = BeautifulSoup(html, 'lxml')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [15]:
title = soup.title
print(title)

<title>Postgraduate programmes - School of Electronic Engineering and Computer Science</title>


In [16]:
text = soup.get_text()
print(soup.text)





document.documentElement.setAttribute('data-js', 'yes');
Postgraduate programmes - School of Electronic Engineering and Computer Science






 

 .filter {
            background-color: #eee;
            margin: 20px 0 0;
            padding: 2%;
        }
        .searchSubmit{
            margin-top: 10px;
        }
        table.display {
            width: 100% !important;
        }
        .content table img {
    	float: right;
    	width: 50%;
    	margin: 0 0 0 20px;
		}
        



  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-2757605-62');



Skip to main content








Global main menu

StudyAboutResearch







Find an expert





Search









Search Queen Mary University London website










Close

















Home




                    Back to home
                








Study


Areas of study

Foundation courses
Biological and biomedical sciences
Business and

In [17]:
# Print the first 10 table rows
rows = soup.find_all('tr')  # the 'tr' tag in html denotes a table row
print(rows[:10])

[<tr><th scope="col">Postgraduate degree programmes</th><th scope="col">Part-time<br/>(2 year)</th><th scope="col">Full-time<br/>(1 year)</th></tr>, <tr>
<td><span>Advanced Electronic and Electrical Engineering</span></td>
<td style="text-align: center;"><a href="https://www.qmul.ac.uk/postgraduate/taught/coursefinder/courses/advanced-electronic-and-electrical-engineering-msc/" title="Use alt + click to follow the link">H60C</a></td>
<td style="text-align: center;"><a href="https://www.qmul.ac.uk/postgraduate/taught/coursefinder/courses/advanced-electronic-and-electrical-engineering-msc/" title="Use alt + click to follow the link">H60A</a></td>
</tr>, <tr>
<td>Artificial Intelligence</td>
<td style="text-align: center;"><a href="https://www.qmul.ac.uk/postgraduate/taught/coursefinder/courses/artificial-intelligence-msc/" title="Use alt + click to follow the link">I4U2</a> </td>
<td style="text-align: center;"><a href="https://www.qmul.ac.uk/postgraduate/taught/coursefinder/courses/arti

In [18]:
for row in rows:
    row_td = row.find_all('td')  # the 'td' tag in html code denotes a table cell
    print(row_td)
type(row_td)

[]
[<td><span>Advanced Electronic and Electrical Engineering</span></td>, <td style="text-align: center;"><a href="https://www.qmul.ac.uk/postgraduate/taught/coursefinder/courses/advanced-electronic-and-electrical-engineering-msc/" title="Use alt + click to follow the link">H60C</a></td>, <td style="text-align: center;"><a href="https://www.qmul.ac.uk/postgraduate/taught/coursefinder/courses/advanced-electronic-and-electrical-engineering-msc/" title="Use alt + click to follow the link">H60A</a></td>]
[<td>Artificial Intelligence</td>, <td style="text-align: center;"><a href="https://www.qmul.ac.uk/postgraduate/taught/coursefinder/courses/artificial-intelligence-msc/" title="Use alt + click to follow the link">I4U2</a> </td>, <td style="text-align: center;"><a href="https://www.qmul.ac.uk/postgraduate/taught/coursefinder/courses/artificial-intelligence-msc/" title="Use alt + click to follow the link">I4U1</a> </td>]
[<td><span>Big Data Science</span></td>, <td style="text-align: center;

bs4.element.ResultSet

In [19]:
str_cells = str(row_td)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
print(cleantext)

[Digital and Technology Solutions (Apprenticeship), I4DA,  ]


In [20]:
# Create an empty list where the table header will be stored
header_list = []

# Find the 'th' html tags which denote table header
col_labels = soup.find_all('th')
col_str = str(col_labels)
cleantext_header = BeautifulSoup(col_str, "lxml").get_text()  # extract the text without HTML tags
header_list.append(cleantext_header) # Add the clean table header to the list

print(header_list)

['[Postgraduate degree programmes, Part-time(2 year), Full-time(1 year)]']


In [21]:
table_list = []

# For every row in the table, find each cell element and add it to the list
for row in rows:
    row_td = row.find_all('td')
    row_cells = str(row_td)
    row_cleantext = BeautifulSoup(row_cells, "lxml").get_text()  # extract the text without HTML tags
    table_list.append(row_cleantext)  # Add the clean table row to the list
    
print(table_list)

['[]', '[Advanced Electronic and Electrical Engineering, H60C, H60A]', '[Artificial Intelligence, I4U2\xa0, I4U1\xa0]', '[Big Data Science, H6J6, H6J7]', '[Computer Games, \xa0, I4U4]', '[Computer Science, G4U2, G4U1]', '[Computer Science by Research, G4Q2, G4Q1]', '[Computing and Information Systems, G5U6, G5U5]', '[Data Science and Artificial Intelligence by Conversion, \xa0, I4U5\xa0]', '[Electronic Engineering by Research, H6T6, H6T5]', '[Internet of Things (Data), I1T2, I1T0]', '[Machine Learning for Visual Data Analytics, H6JZ, H6JE]', '[Sound and Music Computing\xa0, H6T4, H6T8]', '[Telecommunication and Wireless Systems, H6JD, H6JA]', '[Digital and Technology Solutions (Apprenticeship), I4DA, \xa0]']


In [22]:
df_header = pd.DataFrame(header_list)
df_header.head()

,0
0,"[Postgraduate degree programmes, Part-time(2 y..."


In [23]:
df_header2 = df_header[0].str.split(',', expand=True)
df_header2

,0,1,2
0,[Postgraduate degree programmes,Part-time(2 year),Full-time(1 year)]


In [24]:
# We remove uneccessary characters from the header
df_header2[0] = df_header2[0].str.strip('[')
df_header2[2] = df_header2[2].str.strip(']')


In [25]:
df_table = pd.DataFrame(table_list)
df_table2 = df_table[0].str.split(',', expand=True)
df_table2.head(10)

,0,1,2
0,[],None,None
1,[Advanced Electronic and Electrical Engineering,H60C,H60A]
2,[Artificial Intelligence,I4U2,I4U1 ]
3,[Big Data Science,H6J6,H6J7]
4,[Computer Games,,I4U4]
5,[Computer Science,G4U2,G4U1]
6,[Computer Science by Research,G4Q2,G4Q1]
7,[Computing and Information Systems,G5U6,G5U5]
8,[Data Science and Artificial Intelligence by C...,,I4U5 ]
9,[Electronic Engineering by Research,H6T6,H6T5]


In [26]:
# Remove uneccesary characters
df_table2[0] = df_table2[0].str.strip('[')
df_table2[0] = df_table2[0].str.strip(']')
df_table2[2] = df_table2[2].str.strip(']')

# Remove all rows with any missing values
df_table3 = df_table2.dropna(axis=0, how='any')

df_table3

,0,1,2
1,Advanced Electronic and Electrical Engineering,H60C,H60A
2,Artificial Intelligence,I4U2,I4U1
3,Big Data Science,H6J6,H6J7
4,Computer Games,,I4U4
5,Computer Science,G4U2,G4U1
6,Computer Science by Research,G4Q2,G4Q1
7,Computing and Information Systems,G5U6,G5U5
8,Data Science and Artificial Intelligence by Co...,,I4U5
9,Electronic Engineering by Research,H6T6,H6T5
10,Internet of Things (Data),I1T2,I1T0


In [27]:


# We concatenate the two dataframes
frames = [df_header2, df_table3]
df = pd.concat(frames)

df2 = df.rename(columns=df.iloc[0]) # We assign the first row to be the dataframe header
df3 = df2.drop(df2.index[0]) # We drop the replicated header from the first row of the dataframe

df3

,Postgraduate degree programmes,Part-time(2 year),Full-time(1 year)
1,Advanced Electronic and Electrical Engineering,H60C,H60A
2,Artificial Intelligence,I4U2,I4U1
3,Big Data Science,H6J6,H6J7
4,Computer Games,,I4U4
5,Computer Science,G4U2,G4U1
6,Computer Science by Research,G4Q2,G4Q1
7,Computing and Information Systems,G5U6,G5U5
8,Data Science and Artificial Intelligence by Co...,,I4U5
9,Electronic Engineering by Research,H6T6,H6T5
10,Internet of Things (Data),I1T2,I1T0


In [28]:
list1=[] #Creating two lists for part-time and full -time
list2 = []
for row in rows[1:15]:
  try:
    ro=row("td")[1]('a')[0]['href']
    list1.append(ro)
  except:
    list1.append("Not Available") # Append Not available if the href is not available
  try:
    ro=row("td")[2]('a')[0]['href']
    list2.append(ro)
  except:
    list2.append("Not Available")


In [29]:
clm4=pd.DataFrame(list1)
clm5=pd.DataFrame(list2)

In [30]:
frame=[clm4,clm5]
c=pd.concat((frame),axis=1)

In [31]:
c.columns = ["URL for part-time study programme", "URL for full-time study programme"]
c

,URL for part-time study programme,URL for full-time study programme
0,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
1,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
2,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
3,Not Available,https://www.qmul.ac.uk/postgraduate/taught/cou...
4,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
5,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
6,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
7,Not Available,https://www.qmul.ac.uk/postgraduate/taught/cou...
8,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
9,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...


In [33]:
df3.reset_index(drop=True, inplace = True)

In [34]:
frame1=[df3,c]
l=pd.concat((frame1),axis=1)
l

,Postgraduate degree programmes,Part-time(2 year),Full-time(1 year),URL for part-time study programme,URL for full-time study programme
0,Advanced Electronic and Electrical Engineering,H60C,H60A,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
1,Artificial Intelligence,I4U2,I4U1,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
2,Big Data Science,H6J6,H6J7,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
3,Computer Games,,I4U4,Not Available,https://www.qmul.ac.uk/postgraduate/taught/cou...
4,Computer Science,G4U2,G4U1,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
5,Computer Science by Research,G4Q2,G4Q1,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
6,Computing and Information Systems,G5U6,G5U5,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
7,Data Science and Artificial Intelligence by Co...,,I4U5,Not Available,https://www.qmul.ac.uk/postgraduate/taught/cou...
8,Electronic Engineering by Research,H6T6,H6T5,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
9,Internet of Things (Data),I1T2,I1T0,https://www.qmul.ac.uk/postgraduate/taught/cou...,https://www.qmul.ac.uk/postgraduate/taught/cou...
